In [1]:
try:
    indiana
except NameError:
    import os
    os.chdir("..")
    indiana=1

In [2]:
from CloudAtlas.DadosAbertos import CamaraFederal

In [3]:
camara = CamaraFederal()

In [10]:
proposicoes, _ = camara.obter_proposicoes(asdasdasdasd="2021-04-15", datasdadasFim="2021-04-15")

In [11]:
proposicoes

{'status': 400,
 'title': 'Requisição inválida',
 'detail': 'Parâmetro(s) inválido(s).',
 'instance': 'datasdadasFim, asdasdasdasd',
 'code': 5}